In [ ]:
import pytest
import lqrt
import numpy as np
import scipy as sp
import pandas as pd

In [ ]:
from dabest import Dabest
from data.mocked_data_test_01 import dabest_default_kwargs
from data.mocked_data_test_04 import df

In [ ]:
# example of sequential repeated measures
sequential = Dabest(df, id_col = "ID",
                         idx=("First", "Second", "Third", "Fourth", "Fifth"),
                         paired = "sequential",
                         **dabest_default_kwargs)

# example of baseline repeated measures
baseline = Dabest(df, id_col = "ID",
                       idx=("First", "Second", "Third", "Fourth", "Fifth"),
                       paired = "baseline",
                       **dabest_default_kwargs)

test_mean_diff_sequential

In [ ]:
mean_diff = sequential.mean_diff.results['difference'].to_list()
np_result = [np.mean(df.iloc[:,i+1]-df.iloc[:,i]) for i in range(1,5)]
assert mean_diff == pytest.approx(np_result)

test_median_diff_sequential

In [ ]:
median_diff = sequential.median_diff.results['difference'].to_list()
np_result = [np.median(df.iloc[:,i+1]-df.iloc[:,i]) for i in range(1,5)]
assert median_diff == pytest.approx(np_result)

test_mean_diff_baseline

In [ ]:
mean_diff = baseline.mean_diff.results['difference'].to_list()
np_result = [np.mean(df.iloc[:,i]-df.iloc[:,1]) for i in range(2,6)]
assert mean_diff == pytest.approx(np_result)

test_median_diff_baseline

In [ ]:
median_diff = baseline.median_diff.results['difference'].to_list()
np_result = [np.median(df.iloc[:,i]-df.iloc[:,1]) for i in range(2,6)]
assert median_diff == pytest.approx(np_result)

test_cohens_d_sequential

In [ ]:
cohens_d = sequential.cohens_d.results['difference'].to_list()
np_result = [np.mean(df.iloc[:,i+1]-df.iloc[:,i])
                /np.sqrt((np.var(df.iloc[:,i+1], ddof=1)+np.var(df.iloc[:,i], ddof=1))/2) 
            for i in range(1,5)]
assert cohens_d == pytest.approx(np_result)

test_hedges_g_sequential

In [ ]:
from math import gamma
hedges_g = sequential.hedges_g.results['difference'].to_list()
a = 47*2-2
fac = gamma(a/2)/(np.sqrt(a/2)*gamma((a-1)/2))
np_result = [np.mean(df.iloc[:,i+1]-df.iloc[:,i])*fac
                /np.sqrt((np.var(df.iloc[:,i+1], ddof=1)+np.var(df.iloc[:,i], ddof=1))/2) 
            for i in range(1,5)] 
assert hedges_g == pytest.approx(np_result)

test_cohens_d_baseline

In [ ]:
cohens_d = baseline.cohens_d.results['difference'].to_list()
np_result = [np.mean(df.iloc[:,i]-df.iloc[:,1])
                /np.sqrt((np.var(df.iloc[:,i], ddof=1)+np.var(df.iloc[:,1], ddof=1))/2) 
            for i in range(2,6)]
assert cohens_d == pytest.approx(np_result)

test_hedges_g_baseline

In [ ]:
from math import gamma
hedges_g = baseline.hedges_g.results['difference'].to_list()
a = 47*2-2
fac = gamma(a/2)/(np.sqrt(a/2)*gamma((a-1)/2))
np_result = [np.mean(df.iloc[:,i]-df.iloc[:,1])*fac
                /np.sqrt((np.var(df.iloc[:,i], ddof=1)+np.var(df.iloc[:,1], ddof=1))/2) 
            for i in range(2,6)]
assert hedges_g == pytest.approx(np_result)

test_paired_stats_sequential

In [ ]:
np_result = sequential.mean_diff.results
    
p1 = [sp.stats.ttest_rel(df.iloc[:,i], df.iloc[:,i+1], nan_policy='omit').pvalue
            for i in range(1,5)] 
assert np_result["pvalue_paired_students_t"].to_list() == pytest.approx(p1)

p2 = [sp.stats.wilcoxon(df.iloc[:,i], df.iloc[:,i+1]).pvalue
            for i in range(1,5)] 
assert np_result["pvalue_wilcoxon"].to_list() == pytest.approx(p2)

test_paired_stats_baseline

In [ ]:
np_result = baseline.mean_diff.results
    
p1 = [sp.stats.ttest_rel(df.iloc[:,1], df.iloc[:,i], nan_policy='omit').pvalue
            for i in range(2,6)] 
assert np_result["pvalue_paired_students_t"].to_list() == pytest.approx(p1)

p2 = [sp.stats.wilcoxon(df.iloc[:,1], df.iloc[:,i]).pvalue
            for i in range(2,6)] 
assert np_result["pvalue_wilcoxon"].to_list() == pytest.approx(p2)

test_lqrt_paired_sequential

In [ ]:
lqrt_result = sequential.mean_diff.lqrt["pvalue_paired_lqrt"].to_list()
                             
p1 = [lqrt.lqrtest_rel(df.iloc[:,i], df.iloc[:,i+1], random_state=12345).pvalue
            for i in range(1,5)] 

assert lqrt_result == pytest.approx(p1)

test_lqrt_paired_baseline

In [ ]:
lqrt_result = baseline.mean_diff.lqrt["pvalue_paired_lqrt"].to_list()
                             
p1 = [lqrt.lqrtest_rel(df.iloc[:,1], df.iloc[:,i], random_state=12345).pvalue
            for i in range(2,6)] 

assert lqrt_result == pytest.approx(p1)